Import libraries

In [52]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

Crawl and save in dataframe

In [53]:
#tên, năm ra mắt, thể loại, đạo diễn, thời lượng, score, trạng thái, ngôn ngữ, ngân sách, doanh thu
name = []
release_year = []
genre = []
director = []
runtime = []
score = []
status = []
language = []
budget = []
revenue = []

#lấy nội dung html
for page in range(50):
    print('page ' + str(page + 1))
    root_url = 'https://www.themoviedb.org/movie/'
    base_url = root_url + '?page=' + str(page + 1)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(base_url, headers=headers)
    if response.status_code == 200:
        content = response.content.decode('utf-8', errors='replace')
        soup = BeautifulSoup(content, 'html.parser')
    else:
        print('Failed to retrieve web data from the website.')

    #lấy đường dẫn của các movie cụ thể
    set_movie_id = set()
    for a_tag in soup.find_all('a', href=re.compile(r'/movie/')):
        match = re.search(r'/movie/(\d+)', a_tag['href'])
        if match:
            set_movie_id.add(match.group(1))
    list_movie_id = list(set_movie_id)
    
    #truy cập vào các movie cụ thể để lấy data
    for movie_id in list_movie_id:
        movie_url = root_url + movie_id
        response = requests.get(movie_url, headers=headers)
        if response.status_code == 200:
            content = response.content.decode('utf-8', errors='replace')
            soup = BeautifulSoup(content, 'html.parser')
        else:
            print('Failed to retrieve movie data from the website.')
            break
        #name
        name_ = soup.find('div', class_='title').find('a').text
        name.append(name_)

        #release year
        year_ = soup.find('span', class_='tag release_date').text.strip("()") 
        release_year.append(year_)
        
        #genres
        genres = [a.text for a in soup.find('span', class_='genres').find_all('a')]
        genre.append(genres)

        #runtime
        runtime_ = soup.find('span', class_='runtime').text.strip() if soup.find('span', class_='runtime') is not None else ' -'
        runtime.append(runtime_)

        #score
        score_ = soup.find('div', class_='user_score_chart')['data-percent']
        score.append(score_)

        #director
        director_ = soup.find('li', class_='profile').find('a').text if soup.find('li', class_='profile') is not None else ' -'
        director.append(director_)

        #status, language, budget, revenue
        p_section_element = soup.find('section', class_='facts left_column').find_all('p') 
        for ele in p_section_element:
            text = ele.get_text(strip=True)
            if text.startswith('Status'):
                status_ = text.split('Status')[1]
            elif text.startswith('Original Language'):
                language_ = text.split('Original Language')[1]
            elif text.startswith('Budget'):
                budget_ = text.split('Budget')[1]
            elif text.startswith('Revenue'):
                 revenue_ = text.split('Revenue')[1]
        status.append(status_)
        language.append(language_)
        budget.append(budget_)
        revenue.append(revenue_)

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 21
page 22
page 23
page 24
page 25
page 26
page 27
page 28
page 29
page 30
page 31
page 32
page 33
page 34
page 35
page 36
page 37
page 38
page 39
page 40
page 41
page 42
page 43
page 44
page 45
page 46
page 47
page 48
page 49
page 50


In [54]:
movie_df = pd.DataFrame({'Name': name, 'Released year': release_year, 'Genre': genre, 'Director': director, 'Runtime': runtime,
                         'Score': score, 'Status': status, 'Language': language, 'Budget': budget, 'Revenue': revenue})
movie_df

,Name,Released year,Genre,Director,Runtime,Score,Status,Language,Budget,Revenue
0,Reign of Chaos,2022,"[Action, Horror, Fantasy]",Rebecca Matthews,1h 17m,56.0,Released,English,-,-
1,Five Nights at Freddy's,2023,"[Horror, Mystery]",Emma Tammi,1h 50m,78.0,Released,English,"$20,000,000.00","$286,700,000.00"
2,Family Switch,2023,"[Comedy, Fantasy, Family, Crime]",McG,1h 46m,64.0,Released,English,-,-
3,Hell House LLC Origins: The Carmichael Manor,2023,"[Horror, Mystery]",Stephen Cognetti,1h 38m,63.0,Released,English,-,-
4,Leo,2023,"[Animation, Comedy, Family]",Robert Smigel,1h 42m,75.0,Released,English,-,-
...,...,...,...,...,...,...,...,...,...,...
995,Asteroid City,2023,"[Comedy, Drama]",Wes Anderson,1h 45m,66.0,Released,English,"$25,000,000.00","$47,281,670.00"
996,Red Dawn,2012,"[Action, Thriller]",Dan Bradley,1h 33m,57.0,Released,English,"$65,000,000.00","$44,806,783.00"
997,Mount Adams,2021,"[Action, Adventure, Science Fiction]",Phillip Wade,1h 34m,0.0,Released,English,-,-
998,Barcelona Christmas Night,2015,"[Romance, Comedy, Drama]",Dani de la Orden,1h 35m,63.0,Released,Catalan; Valencian,-,-


In [55]:
movie_df.to_csv('movie.csv', index=False)